In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

from amftrack.pipeline.launching.run_super import (
    run_launcher,
    directory_scratch,
    directory_project,
    directory_project,
    run_parallel_stitch,
    run_parallel_transfer,
)
import os
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    update_analysis_info,
    get_analysis_info,
    get_current_folders,
    get_folders_by_plate_id,
)

from time import time_ns
from amftrack.util.dbx import upload_folders, load_dbx, download, get_dropbox_folders_prince
from datetime import datetime
from amftrack.pipeline.launching.run_super import (
    run_parallel,
    directory_scratch,
    directory_project,
    run_parallel_stitch,
)
from amftrack.util.dbx import read_saved_dropbox_state, get_dropbox_folders_prince
import sys
import os

from amftrack.util.sys import get_dirname, temp_path
import pandas as pd
import ast
from scipy import sparse
import scipy.io as sio
import cv2
import imageio.v2 as imageio
import numpy as np
import scipy.sparse
import os
from time import time
from amftrack.pipeline.functions.image_processing.extract_skel import (
    extract_skel_new_prince,
    run_back_sub,
    bowler_hat,
)

from amftrack.util.sys import get_dates_datetime, get_dirname
import shutil
import matplotlib.pyplot as plt
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    Experiment,
    save_graphs,
    load_graphs,
    Edge,
    Node,
)
from amftrack.pipeline.functions.image_processing.experiment_util import (
    get_random_edge,
    distance_point_edge,
    plot_edge,
    plot_edge_cropped,
    find_nearest_edge,
    get_edge_from_node_labels,
    plot_full_image_with_features,
    get_all_edges,
    get_all_nodes,
    find_neighboring_edges,
    reconstruct_image,
    reconstruct_skeletton_from_edges,
    reconstruct_skeletton_unicolor,
    reconstruct_image_from_general,
    plot_full,
    plot_edge_color_value,
)
from matplotlib import cm
import networkx as nx

In [ ]:
directory_targ = directory_project
directory_targ = directory_targ+'fig1/'

In [ ]:

update_plate_info(directory_targ, local=True)
all_folders = get_current_folders(directory_targ, local=True)

In [ ]:
folders = all_folders.loc[all_folders["/Analysis/nx_graph_pruned_width.p"] == True]
len(folders),len(all_folders)

In [ ]:
plates = all_folders['unique_id'].unique()
results = {}
for plate in plates:
    folders = all_folders.loc[all_folders["unique_id"] == plate]
    folders = folders.loc[folders["/Analysis/nx_graph_pruned_width.p"] == True]
    folders = folders.sort_values(by="datetime")

    exp = Experiment(directory_targ)
    i = 0
    exp.load(folders.iloc[i : i + 1], suffix="_width")
    for t in range(exp.ts):
        exp.load_tile_information(t)
    G = exp.nx_graph[t]
    components = nx.connected_components(G)

    # Find the largest connected component
    largest_component = max(components, key=len)

    # Create a new graph representing the largest connected component
    largest_component_graph = G.subgraph(largest_component)
    exp.nx_graph[t] = largest_component_graph
    edges = get_all_edges(exp, t)
    lengths = [edge.length_um(t) for edge in edges]
    widths = [edge.width(t) for edge in edges]
    folders = all_folders.loc[all_folders["unique_id"] == plate]
    strain = folders["strain"].value_counts().idxmax()
    if np.max(widths)!=40:
        results[plate] = edges, lengths, widths,[strain]*len(edges)
    else:
        print(plate)

In [ ]:
data = []

# Iterate over the results to create rows for the DataFrame
for plate, (edges, lengths, widths, strains) in results.items():
    for edge, length, width, strain in zip(edges, lengths, widths, strains):
        data.append({'plate': plate, 'edge': edge, 'length': length, 'width': width, 'strain': strain})

# Create DataFrame from the list
df_results = pd.DataFrame(data)


In [ ]:
df_results['radius'] = df_results['width']/2
df_results['weight'] = df_results['radius']**2*df_results['length']
# df_results['weight'] = df_results['length']
df_results.loc[df_results['strain']=="'A5sp3'","strain"]="'A5'"
df_results['strain'].unique()

In [ ]:
import seaborn as sns
plt.style.use('presentation.mplstyle')
import logging

# Set the logging level to WARNING to suppress INFO and DEBUG messages
logging.basicConfig(level=logging.WARNING)

In [ ]:
palette = {"'C2'": (0.17058823529411765, 0.4946558433997788, 0.9667184042691874),
 "'A5'": (0.16666666666666663, 0.8660254037844386, 0.8660254037844387),
 "'Agg'": (0.503921568627451, 0.9999810273487268, 0.7049255469061472),
 "'Clarum'": (0.8333333333333333, 0.8660254037844387, 0.5000000000000001),
 "'C3'": (1.0, 0.4946558433997788, 0.2558427775944356)}



In [ ]:
plt.close('all')
from matplotlib.ticker import FuncFormatter
cm = 1/2.54  # centimeters in inches

fig,ax = plt.subplots(figsize = (6*cm,4*cm))
select = df_results[df_results['width']>0.1]
select = select[select['width']<=14]

# sns.histplot(select,x = 'radius',hue="strain",kde = True, weights = "weight",stat="density",bins = 30, common_norm=False,palette= palette,fill=False)
for strain in select['strain'].unique():
    strain_data = select[select['strain'] == strain]
    weighted_mean = (strain_data['radius'] * strain_data['weight']).sum() / strain_data['weight'].sum()
    ax.axvline(weighted_mean, color=palette[strain], linestyle='--',linewidth = 2,alpha = 0.5)
    sns.kdeplot(strain_data,x='radius', ax=ax, label=strain, weights = "weight",color = palette[strain],alpha = 0.9)
ax.set_xlabel('radius ($\mu m$)')
ax.set_ylabel('density')
ax.set_yscale('log')
# ax.set_xscale('log')

ax.set_ylim(1e-2,1)
ax.set_xlim(0.1,7)

ax.set_xticks([1,2, 3, 4,5,6, 7])
ax.set_xticklabels(['1','2', '3', '4','5','6', '7'])
# def format_fn(tick_val, tick_pos):
#     if int(tick_val) in [1,2, 4, 6, 7]:
#         return str(int(tick_val))
#     else:
#         return ''

# # Apply the formatter to the x-axis
# ax.xaxis.set_major_formatter(FuncFormatter(format_fn))
plt.savefig(os.path.join('figures','Figure1D2.pdf'), transparent=True, bbox_inches='tight')


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Your existing code for filtering the DataFrame
fig, ax = plt.subplots()
select = df_results[(df_results['radius'] > 0) & (df_results['radius'] <= 14)]

# Define a color palette for the strains

# Create a violin plot
sns.violinplot(x='strain', y='radius', data=select, palette=palette, scale='width')

# # Calculate and plot the weighted mean for each strain
# for strain in select['strain'].unique():
#     strain_data = select[select['strain'] == strain]
#     weighted_mean = (strain_data['radius'] * strain_data['weight']).sum() / strain_data['weight'].sum()
#     ax.axvline(weighted_mean, color=palette[strain], linestyle='--')

# Optional: Adjusting the layout for better visibility
ax.set_ylim(0,7)
plt.tight_layout()

plt.show()
